In [13]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

In [22]:
def model(p_W, p_H):
    weather = yield tfp.distributions.JointDistributionCoroutine.Root(tfp.distributions.Bernoulli(probs=p_W, name='weather'))
    weather_to_happy = p_H#tf.constant([0.6, 0.9])
    terminal = yield tfp.distributions.Bernoulli(probs=weather_to_happy[weather], name='happiness')


In [23]:
model_joint = tfp.distributions.JointDistributionCoroutineAutoBatched(lambda: model(tf.constant(0.3), tf.constant([0.6, 0.9])))
model_joint

<tfp.distributions.JointDistributionCoroutineAutoBatched 'JointDistributionCoroutineAutoBatched' batch_shape=[] event_shape=StructTuple(
  weather=[],
  happiness=[]
) dtype=StructTuple(
  weather=int32,
  happiness=int32
)>

In [25]:
data = model_joint.sample(100)
data

StructTuple(
  weather=<tf.Tensor: shape=(100,), dtype=int32, numpy=
    array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
           0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
           0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
           0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
           0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])>,
  happiness=<tf.Tensor: shape=(100,), dtype=int32, numpy=
    array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
           1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
           0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1])>
)

Take the model defined above as the generator. We now want to do parameter estimation using the data we sampled. 

$$
\log\mathcal L(D) = \sum_{i=0}^{N-1}\log\underbrace{p(W=w_i, H=h_{i})}_{p(w_i)p(h_i|w_i)}
$$
Notice that we know $p(w_i)p(h_i|w_i)$ to be $Ber_{w_i}(\theta_w)Ber_{h_i}(\vec\theta_{h_{i}})$. Then, 
$$
\theta_W,\vec\theta_H=\arg\max_{\theta_W,\theta_H\in[0,1]^{1,2}}\log\mathcal L(D;\theta_W,\theta_H)
$$
We're going to get the derivative using auto grad

In [28]:
theta_W_fit = tfp.util.TransformedVariable(0.5, bijector=tfp.bijectors.SoftClip(low=0.0, high=1.0))
theta_H_fit = tfp.util.TransformedVariable([0.5, 0.5], bijector=tfp.bijectors.SoftClip(low=0.0, high=1.0))

In [29]:
model_joint_fit =  tfp.distributions.JointDistributionCoroutineAutoBatched(lambda: model(theta_W_fit, theta_H_fit))
model_joint_fit

<tfp.distributions.JointDistributionCoroutineAutoBatched 'JointDistributionCoroutineAutoBatched' batch_shape=[] event_shape=StructTuple(
  weather=[],
  happiness=[]
) dtype=StructTuple(
  weather=int32,
  happiness=int32
)>

In [30]:
model_joint_fit.log_prob(data)

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862944, -1.3862944, -1.3862944, -1.3862944, -1.3862944,
       -1.3862

In [31]:
neg_log_ll = lambda: -tf.reduce_sum(model_joint_fit.log_prob(data))

In [32]:
tfp.math.minimize(loss_fn=neg_log_ll, optimizer=tf.optimizers.Adam(0.01), num_steps=1000)

<tf.Tensor: shape=(1000,), dtype=float32, numpy=
array([138.62944 , 138.14899 , 137.67331 , 137.20251 , 136.73666 ,
       136.27583 , 135.82007 , 135.36946 , 134.92407 , 134.48393 ,
       134.04912 , 133.61969 , 133.19568 , 132.77711 , 132.36406 ,
       131.9565  , 131.5545  , 131.15807 , 130.76723 , 130.38196 ,
       130.00232 , 129.62828 , 129.25986 , 128.89702 , 128.53978 ,
       128.1881  , 127.841995, 127.501396, 127.166336, 126.83673 ,
       126.512566, 126.19383 , 125.880455, 125.57241 , 125.26964 ,
       124.97209 , 124.67975 , 124.39252 , 124.11039 , 123.833275,
       123.56111 , 123.29386 , 123.031456, 122.77382 , 122.520905,
       122.27266 , 122.028984, 121.78984 , 121.555145, 121.324844,
       121.09888 , 120.877144, 120.6596  , 120.44618 , 120.23681 ,
       120.03142 , 119.82994 , 119.6323  , 119.438446, 119.24831 ,
       119.061806, 118.878876, 118.69947 , 118.5235  , 118.350914,
       118.181656, 118.01564 , 117.85281 , 117.693115, 117.53649 ,
       117.38

In [35]:
print( theta_W_fit.numpy(), theta_H_fit.numpy())

0.24000001 [0.67105246 0.9539699 ]


In [38]:
def model_1(w_prob=tf.constant(0.8), h_prob=tf.constant([0.7,0.95])):
    weather = yield tfp.distributions.JointDistributionCoroutine.Root(tfp.distributions.Bernoulli(probs=w_prob, name="W"))
    mood = yield tfp.distributions.Bernoulli(probs=h_prob, name='h')
    
joint_1 = tfp.distributions.JointDistributionCoroutineAutoBatched(lambda: model_1())
joint_1 # Non-plated

<tfp.distributions.JointDistributionCoroutineAutoBatched 'JointDistributionCoroutineAutoBatched' batch_shape=[] event_shape=StructTuple(
  W=[],
  head=[2]
) dtype=StructTuple(
  W=int32,
  head=int32
)>

In [39]:
data = joint_1.sample(10)

In [40]:
print('likelihood: ', tf.reduce_prod(joint_1.prob(data)))

likelihood:  tf.Tensor(1.3960019e-07, shape=(), dtype=float32)


In [43]:
def model_2(w_prob=tf.constant(0.8), h_prob=tf.constant([0.7,0.95]), n_days=10):
    weather = yield tfp.distributions.JointDistributionCoroutine.Root(tfp.distributions.Bernoulli(probs=w_prob*tf.ones(n_days), name="W"))
    mood = yield tfp.distributions.Bernoulli(probs=tf.where(weather==1, h_prob[1], h_prob[0]), name='h')

In [49]:
joint_2 = tfp.distributions.JointDistributionCoroutineAutoBatched(lambda: model_2())
joint_2.sample()

StructTuple(
  W=<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 0, 1, 1, 1, 1, 1, 0, 1, 1])>,
  h=<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 0, 1, 1, 1, 1, 1, 0, 1, 1])>
)

In [69]:
def model_3(alpha=tf.constant(10.0), beta=tf.constant(5.0), h_prob=tf.constant([0.7, 0.95]), n_days=10):
    theta_RV = yield tfp.distributions.JointDistributionCoroutine.Root(tfp.distributions.Beta(alpha, beta, name='weather_param'))
    weather = yield tfp.distributions.Bernoulli(probs=theta_RV*tf.ones(n_days) , name='W')
    mood = yield tfp.distributions.Bernoulli(probs=tf.where(weather==1, h_prob[1], h_prob[0]), name='H')

In [70]:
joint_3 = tfp.distributions.JointDistributionCoroutineAutoBatched(lambda: model_3())


In [72]:
joint_3.sample()

StructTuple(
  weather_param=<tf.Tensor: shape=(), dtype=float32, numpy=0.42909604>,
  W=<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1])>,
  H=<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>
)